In [4]:
from datasets import load_dataset
dataset = load_dataset("multi_news", split = 'test')
df = dataset.to_pandas().sample(2000,random_state = 42)
df.head()

,document,summary
4830,Tweet with a location \n \n You can add locati...,– Denis Finley has taken to Twitter to call Po...
1255,CNN host Piers Morgan just called to discuss h...,– CNN's Piers Morgan thinks gun-rights propone...
80,White House communications director Anthony Sc...,– New White House communications director Anth...
3044,CLOSE Scientists say they've found archaeologi...,– Scientists say they have the first physical ...
4486,Click image above to view graphic \n \n Althou...,– Scientists are calling it a breakthrough and...


In [17]:
from sentence_transformers import SentenceTransformer, util
import torch

In [11]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [12]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [13]:
passage_embeddings = list(model.encode(df['summary'].to_list(), show_progress_bar = True))
passage_embeddings[0].shape

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

(384,)

In [14]:
def find_relevant_news(query):
    query_embedding = model.encode(query)
    query_embedding.shape
    
    similarities = util.cos_sim(query_embedding, passage_embeddings)
    similarities.shape
    
    top_indices = torch.topk(similarities.flatten(), 3).indices
    
    top_relevant_passages = [df.iloc[x.item()]['summary'][:200] + "...." for x in top_indices]
    
    return top_relevant_passages 

In [18]:
find_relevant_news("Find me some articles about technology and artifical intelligence")

C:\Users\HP\anaconda3\lib\site-packages\sentence_transformers\util.py:39: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:264.)
  b = torch.tensor(b)


["– If brain size relative to body size determines IQ, the venerable shrew would be the smartest creature on the planet. But it doesn't, and it's not, and scientists from Austria, the Netherlands, and G....",
 '– Are you a "digital native" or a "digital immigrant," and does it make a difference? Research recently published in the Teaching and Teacher Education journal indicates the concept of so-called digit....',
 "– Using methods borrowed from Google, a group of researchers has analyzed all Wikipedia pages and determined that, at least on the English language version of the site, Frank Sinatra is the world's mo...."]

In [19]:
find_relevant_news("Natural Disasters")

['– The tsunami that killed hundreds, possibly thousands of people after an earthquake in Indonesia on Friday was much bigger and more devastating than would normally be expected after that kind of quak....',
 '– A sad milestone out of Japan: Two weeks after the quake struck, its official death toll has broken the 10,000 mark—and that number is still on the rise, with more than 17,400 missing. Police estimat....',
 '– When you live near a major dam, the last thing you want to hear is that the integrity of it has been "compromised" by landslides. But that\'s exactly what residents living below North Carolina\'s Lake....']

In [20]:
find_relevant_news("Law enforcement and police")

['– The war of words between Chicago and the federal government over "sanctuary cities" policy is heating up. Attorney General Jeff Sessions slammed the city\'s leaders Monday over their lawsuit against ....',
 '– Greg Barnes was in a hurry to get home on Friday, so when he saw police lights behind him on State Road 332 in Muncie, Indiana, "immediately I knew I was in the wrong," he tells WISH. What followed ....',
 '– "We are not thugs. We are professionals," says the leader of a black policing group, addressing a speech in which President Trump urged officers to not be "too nice" when arresting suspects. Perry T....']

In [21]:
find_relevant_news("Politics, diplomacy and nationalism")

['– Tunisia got a new interim president today—parliamentary leader Fouad Mebazaa—who is technically the third leader in 24 hours, notes the Los Angeles Times. He takes over from the previous interim pre....',
 '– The global relief effort in Haiti has devolved into a nasty power struggle, with countries and aid agencies furious at the US takeover of emergency operations. France, Brazil, which runs the UN peac....',
 '– Like a game of Risk, Russia and NATO have not only been suspiciously eying each other in recent months, but also conducting military exercises in border areas that the Washington Post says are makin....']